In [1]:
import html
from urllib.request import Request,urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json 

In [2]:
base_url = 'https://www.epicurious.com/search?content=recipe&page='
prefix_url = 'https://www.epicurious.com'

recipe_titles = []
recipe_img_links = []
recipe_ingredients = []
recipe_instructions = []


In [4]:
# Loop through the pages
cur_count=5228
count=1
for i in range(294,500):  # 2253
    print("Page " + str(i) + " scraping started!!!")
    
    url = base_url + str(i)
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_soup = BeautifulSoup(webpage, "html.parser")
    
    recipe_links_on_page = []
    
    recipe_links = page_soup.find_all('h4', {'class': 'hed'})
#     print(recipe_links)
    

    for link in recipe_links:
        recipe_links_on_page.append(prefix_url + link.find('a')['href'])
        
        # Convert the Beautiful Soup object to string
        link_html = str(link)  
        
        # Use BeautifulSoup on the HTML string
        soup = BeautifulSoup(link_html, 'html.parser')  

        # Finding the <a> tag within <h4> tag
        a_tag = soup.find('a')

        # Extracting text from the <a> tag
        if a_tag:
            recipe_name = a_tag.text
            recipe_titles.append(recipe_name)
        else:
            # If <a> tag not found, add "N/A"
            recipe_titles.append("N/A")  

# print(recipe_links_on_page)
    for recipe_url in recipe_links_on_page:
        req_recipe = Request(recipe_url, headers={'User-Agent': 'Mozilla/5.0'})
        webpage_recipe = urlopen(req_recipe).read()
        page_soup_recipe = BeautifulSoup(webpage_recipe, "html.parser")
        
        # Ingredients
        ingredients = page_soup_recipe.find('div', class_='List-iSNGTT ljAYJm').find_all('div')
        temp_ingredients = [ingredient.text.strip() for ingredient in ingredients]
        recipe_ingredients.append(temp_ingredients)
#         print(temp_ingredients)


        # Recipe Instructions
        instructions = page_soup_recipe.find('div', class_='InstructionsWrapper-hZXqPx RmryN').find('ol').find('li').find_all('p')
        temp_instructions = [instruction.text.strip() for instruction in instructions]
        recipe_instructions.append(temp_instructions)
#         print(recipe_intructions)

        # Creating JSON object for individual recipe
        entry = {
            'Recipe_number': cur_count,
            'Recipe_name': recipe_titles[count-1],
            'Recipe_ingredients': temp_ingredients,
            'Recipe_instructions': temp_instructions,
#             'Recipe_img_link': temp_img
        }

        # Append to JSON file
        with open("dataset_epi.json", mode='a', encoding='utf-8') as feedsjson:
            json.dump(entry, feedsjson)
            feedsjson.write('\n')
        cur_count+=1
        count+=1
        
#         print("Recipe " + str(len(recipe_titles)) + " Scrapped")
    print("Page "+str(i)+" Scraped!!!")





Page 215 scraping started!!!
Page 215 Scraped!!!
Page 216 scraping started!!!
Page 216 Scraped!!!
Page 217 scraping started!!!
Page 217 Scraped!!!
Page 218 scraping started!!!
Page 218 Scraped!!!
Page 219 scraping started!!!
Page 219 Scraped!!!
Page 220 scraping started!!!
Page 220 Scraped!!!
Page 221 scraping started!!!
Page 221 Scraped!!!
Page 222 scraping started!!!
Page 222 Scraped!!!
Page 223 scraping started!!!
Page 223 Scraped!!!
Page 224 scraping started!!!
Page 224 Scraped!!!
Page 225 scraping started!!!
Page 225 Scraped!!!
Page 226 scraping started!!!
Page 226 Scraped!!!
Page 227 scraping started!!!
Page 227 Scraped!!!
Page 228 scraping started!!!
Page 228 Scraped!!!
Page 229 scraping started!!!
Page 229 Scraped!!!
Page 230 scraping started!!!
Page 230 Scraped!!!
Page 231 scraping started!!!
Page 231 Scraped!!!
Page 232 scraping started!!!
Page 232 Scraped!!!
Page 233 scraping started!!!
Page 233 Scraped!!!
Page 234 scraping started!!!
Page 234 Scraped!!!
Page 235 scraping st

KeyboardInterrupt: 

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import json

# Define base_url, prefix_url, and other variables here

count = 1

# Create a retry session
retries = Retry(total=3, backoff_factor=0.5, status_forcelist=[502, 503, 504])
session = requests.Session()
session.mount('http://', HTTPAdapter(max_retries=retries))
session.mount('https://', HTTPAdapter(max_retries=retries))

for i in range(1, 2254):  # 91-2254
    print("Page " + str(i) + " scraping started!!!")
    
    url = base_url + str(i)
    
    try:
        response = session.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=30)
        soup = BeautifulSoup(response.content, "html.parser")
        
        recipe_links_on_page = []
        recipe_links = soup.find_all('h4', {'class': 'hed'})

        for link in recipe_links:
            recipe_links_on_page.append(prefix_url + link.find('a')['href'])
            recipe_name = link.find('a').text.strip()
            recipe_titles.append(recipe_name if recipe_name else "N/A")

        for recipe_url in recipe_links_on_page:
            try:
                response_recipe = session.get(recipe_url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=30)
                soup_recipe = BeautifulSoup(response_recipe.content, "html.parser")
                
                # Ingredients
                ingredients = soup_recipe.find('div', class_='List-iSNGTT ljAYJm').find_all('div')
                temp_ingredients = [ingredient.text.strip() for ingredient in ingredients]
                recipe_ingredients.append(temp_ingredients)

                # Recipe Instructions
                instructions = soup_recipe.find('div', class_='InstructionsWrapper-hZXqPx RmryN').find('ol').find('li').find_all('p')
                temp_instructions = [instruction.text.strip() for instruction in instructions]
                recipe_instructions.append(temp_instructions)

                # Creating JSON object for individual recipe
                entry = {
                    'Recipe_number': count,
                    'Recipe_name': recipe_titles[count - 1],
                    'Recipe_ingredients': temp_ingredients,
                    'Recipe_instructions': temp_instructions,
                }

                # Append to JSON file
                with open("dataset_epi.json", mode='a', encoding='utf-8') as feedsjson:
                    json.dump(entry, feedsjson)
                    feedsjson.write('\n')
                count += 1
                
            except (requests.HTTPError, requests.ConnectionError, requests.Timeout) as e:
                print(f"Error fetching recipe URL: {recipe_url}. Error: {e}")
            
        print("Page " + str(i) + " Scraped!!!")

    except (requests.HTTPError, requests.ConnectionError, requests.Timeout) as e:
        print(f"Error fetching page URL: {url}. Error: {e}")



In [ ]:
# Read json data

import json

data = []
with open('dataset_epi.json', 'r', encoding='utf-8') as json_file:
    for line in json_file:
        try:
            item = json.loads(line)
            data.append(item)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Now 'data' contains all the valid JSON objects
print(data)